In [1]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import re
import json 
import string
import torch
import nltk
import gensim
from nltk import WordNetLemmatizer
from sklearn.metrics import accuracy_score, average_precision_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torchtext
from torchtext.data import get_tokenizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# Carga de datos
Conjunto de datos para entrenar el modelo, con respuestas básicas

In [3]:
f = open('/content/Intent.json')
intents = json.load(f)

# Identificación de variable objetivo y características (preprocesamiento)
Los datos tienen diferentes _intenciones_ y respuestas de acuerdo a la intención

In [4]:
def preprocesamiento(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [10]:
# listas para las palabras tokenizadas, las clases que identifican a cada respuesta, las características y la variable objetivo, en ese orden respectivamente
w = {}
clases = []
X = []
y = []

for intent in intents['intents']:
  if intent['intent'] not in clases:
        clases.append(intent['intent'])
  if intent['intent'] not in w:
      w[intent['intent']] = []
      
  for text in intent['text']:
      X.append(preprocesamiento(text))
      y.append(intent['intent'])
      
  for response in intent['responses']:
      w[intent['intent']].append(response)

# w = [lemmatizer.lemmatize(word.lower()) for word in w if word not in string.punctuation]
# w = sorted(set(w))
# clases = sorted(set(clases))

# Creación de la representación del lenguaje
De a cuerdo a las restricciones del reto, este modelo es desde cero, por lo tanto usamos una bolsa de palabras(BOW), una técnica de representación de texto que crea un vector con las frecuencias de las palabras

In [15]:
training = []
lista = [0] * len(clases)
lemmatizer = WordNetLemmatizer()

for i, doc in enumerate(X):
  bow = []
  txt = lemmatizer.lemmatize(doc.lower())
  # print(txt)
  for word in w:
    bow.append(1) if word not in txt else bow.append(0)

  out = list(lista)
  out[clases.index(y[i])] = 1
  training.append([bow, out])

np.random.shuffle(training)
training = np.array(training, dtype=object)

X_train = torch.FloatTensor(list(training[:, 0]))
y_train = torch.FloatTensor(list(training[:, 1]))

In [17]:
X_train.shape, y_train.shape

(torch.Size([143, 22]), torch.Size([143, 22]))

In [ ]:
# red con una capa oculta y dropout para evitar overfitting
class NeuralNetwork(torch.nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.layers = nn.Sequential(
      nn.Linear(len(X_train[0]), 128),
      nn.Dropout(0.5),
      nn.ReLU(),
      nn.Linear(128, 22),
      nn.Dropout(0.5),
      nn.Softmax()
    )

  def forward(self, X):
    return self.layers(X)

In [ ]:
mlp = NeuralNetwork()

In [ ]:
loss =  nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.01)

In [ ]:
# entrenamiento de la red neuronal
for i in range(1000):
  output = mlp(X_train)
  costo = loss(output, y_train)
  optimizer.zero_grad() # reiniciar el gradiente del optimizador
  costo.backward() # gradiente de la función de costo
  optimizer.step()
  if i % 100 == 0:
    print(f'costo: {costo}')

  with torch.no_grad(): 
    pred_probab = nn.Softmax(dim=1)(mlp(X_train)) 
    y_pred = pred_probab.argmax(1) 
    # acc = average_precision_score(y_train, y_pred) 
    # if i % 100 == 0:
    #   print(f'Accuracy: {acc}')

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


costo: 3.0880229473114014
costo: 3.0491151809692383
costo: 3.0044944286346436
costo: 3.006531238555908
costo: 2.9907784461975098
costo: 2.995671510696411
costo: 2.984285593032837
costo: 3.0149078369140625
costo: 2.996169090270996
costo: 3.0044312477111816


In [ ]:
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, voc):
  tok = clean_text(text)
  bow = [0] * len(voc)
  for w in tok:
    for i, w in enumerate(voc):
      if word == w:
        bow[i] = 1
  return np.array(bow)